In [12]:
import dspy
import os
import requests
import openai

from dotenv import load_dotenv

colbertv2 = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")

In [13]:
load_dotenv()

url = "https://ai.nutanix.com/api/v1/chat/completions" 


openai.api_key= os.environ['OPENAI_API_KEY']
nai_api_key = os.getenv("NAI_API_KEY")

def get_response_llm(prompt):
    response = requests.post(url, 
                         headers={
                                "Authorization": f"Bearer {nai_api_key}", 
                                "accept": "application/json",
                                "Content-Type": "application/json"
                            }, 
                         json={
                                "model": "vllm-llama-3-1",
                                "messages": [
                                    {
                                        "role": "user",
                                        "content": f"{prompt}"
                                    }
                                ],
                                "max_tokens": 256,
                                "stream": False
                            }, 
                         verify=False)
    return response

In [14]:
from langchain.globals import set_llm_cache
from langchain_community.cache import SQLiteCache
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

set_llm_cache(SQLiteCache(database_path="cache.db"))

def retrieve(inputs):
    return [doc["text"] for doc in colbertv2(inputs["question"], k=5)]

In [5]:
colbertv2("cycling")

[{'text': 'Cycling | Cycling, also called bicycling or biking, is the use of bicycles for transport, recreation, exercise or sport. Persons engaged in cycling are referred to as "cyclists", "bikers", or less commonly, as "bicyclists". Apart from two-wheeled bicycles, "cycling" also includes the riding of unicycles, tricycles, quadracycles, recumbent and similar human-powered vehicles (HPVs).',
  'pid': 2201868,
  'rank': 1,
  'score': 27.078739166259766,
  'prob': 0.3544841299722533,
  'long_text': 'Cycling | Cycling, also called bicycling or biking, is the use of bicycles for transport, recreation, exercise or sport. Persons engaged in cycling are referred to as "cyclists", "bikers", or less commonly, as "bicyclists". Apart from two-wheeled bicycles, "cycling" also includes the riding of unicycles, tricycles, quadracycles, recumbent and similar human-powered vehicles (HPVs).'},
 {'text': 'Cycling (ice hockey) | In ice hockey, cycling is an offensive strategy that moves the puck along 

# Normal LCEL
First, let's create a simple RAG pipeline with LCEL like we would normally.

For illustration, let's tackle the following task.

**Task:** Build a RAG system for generating tuning the prompt.

- **Input:** A factual question, which may be fairly complex.

- **Output:** A refinned and tuned prompt 



In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate.from_template(
    "Given {context}, answer the question `{question}` as a better prompt"
)

vanilla_chain = (
    RunnablePassthrough.assign(context=retrieve) | prompt | llm | StrOutputParser()
)

In [18]:
# From DSPy, import the modules that know how to interact with LangChain LCEL.
from dspy.predict.langchain import LangChainModule, LangChainPredict

# This is how to wrap it so it behaves like a DSPy program.
# Just Replace every pattern like `prompt | llm` with `LangChainPredict(prompt, llm)`.
zeroshot_chain = (
    RunnablePassthrough.assign(context=retrieve)
    | LangChainPredict(prompt, llm)
    | StrOutputParser()
)
# Now we wrap it in LangChainModule
zeroshot_chain = LangChainModule(
    zeroshot_chain
)  # then wrap the chain in a DSPy module.

In [19]:
with open('prompts/prompt_01.prompt', 'r') as file:
    base_prompt = file.read()

print(base_prompt)

Analyze the following sales report and provide a summary of the top 3 performing products along with their respective revenue


In [20]:
zeroshot_chain.invoke({"question": base_prompt})

' Based on the provided sales report, please summarize the top 3 performing products and their corresponding revenue.'